## 影评文本分类

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import tensorflow_hub as tfhub
import tensorflow_datasets as tfds

print("Version:", tf.__version__)
print("Eager mode:", tf.executing_eagerly())
print("Hub version:", tfhub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version: 2.0.0-alpha0
Eager mode: True
Hub version: 0.4.0
GPU is NOT AVAILABLE


### 下载数据

In [ ]:
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name = "imdb_reviews",
    split = (train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

### 探索数据